#### **This notebook takes the mean of embedding for each user**

In [10]:
import pandas as pd
import numpy as np
import os
import importlib
import config.config as config_hp
import torch

#### **Load data**

In [2]:
importlib.reload(config_hp)

config = config_hp.config()

embedding_path = config['EMBEDDINGS_PATH']
reply_multilanguage_embedding = embedding_path['reply_multilanguage_embedding']

df_embedding = pd.read_pickle(reply_multilanguage_embedding)

#### **Find the mean**

In [3]:
df_embedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2673091 entries, 0 to 2673090
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   replier_tweetid  object
 1   poster_userid    object
 2   poster_tweetid   object
 3   replier_userid   object
 4   embeddings       object
dtypes: object(5)
memory usage: 102.0+ MB


In [4]:
df_embedding['replier_userid'].nunique()

1123831

In [40]:
type(df_embedding['embeddings'][0])

torch.Tensor

In [ ]:
def get_mean_of_embedding(df):
    '''
    Gets the mean of embedding of the replies for each user
    :param df: dataframe
    
    :return dataframe
    '''
    print(df['replier_userid'].nunique())
    
    df_grp = df.groupby(['replier_userid'])['embeddings'].apply(
        lambda x: torch.stack(x.values.tolist())  #if len(x) > 1 else torch.tensor(x.values[0])
    ).reset_index()
    
    df_grp['mean_tensor'] = df_grp['embeddings'].apply(
        lambda x: torch.mean(x, dim=0)
    )

    importlib.reload(config_hp)

    config = config_hp.config()

    embedding_path = config['EMBEDDINGS_PATH']
    mean_embedding_user = embedding_path['mean_embedding_user']
    
    df_grp.to_pickle(mean_embedding_user)
    
    return df_grp
    
    
df_grp = get_mean_of_embedding(df_embedding)

1123831


In [53]:
df_grp['mean_tensor'][0].shape

torch.Size([1, 768])

In [54]:
df_grp['embeddings'][0].shape

torch.Size([8, 1, 768])